## 1. 검색 성능 평가

- 구축한 검색 시스템(Retriever)이 얼마나 잘 동작하는지 정량적으로 평가하는 것은 매우 중요함. 
- 이를 위해 테스트 데이터셋을 만들고, 정보 검색(IR) 분야의 평가지표를 사용함.

### 1.1 테스트 데이터 준비

- 좋은 평가를 위해서는 양질의 테스트 데이터셋이 필요함. 
- 여기서는 기존 문서들을 기반으로 질문-답변(QA) 쌍을 합성하고, 이를 검토/수정하여 사용함.

`(1) 평가용 문서 데이터 정제`

- `korean_docs`를 평가용으로 좀 더 가공함.
  - 각 문서에 고유 `doc_id` 메타데이터 추가.
  - 문장 구분 기호를 줄바꿈 문자로 변경 (가독성 및 LLM 처리 용이성).
  - 문서 내용 끝에 해당 문서가 어떤 기업에 대한 정보인지 명시 (LLM이 QA 생성 시 참고하도록).
- JSONL (JSON Lines) 형식으로 저장하여 재사용 용이하게 함.
- **장점:** 평가 데이터의 일관성 및 재현성 확보, LLM이 QA 생성 시 컨텍스트 명확화.
- **단점:** 수동 전처리 과정 필요, 데이터 형식 변환에 따른 약간의 오버헤드.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader
from glob import glob
import os 

In [3]:
def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        print(f"로딩 중: {text_file}")
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

In [4]:
korean_txt_files = glob(os.path.join('data', '*_KR.txt'))
if not korean_txt_files:
    print("'data' 폴더에 '*_KR.txt' 파일이 없습니다. 예시 데이터를 생성하거나 경로를 확인하세요.")
    korean_data = [] # 빈 리스트로 초기화
else:
    korean_data = load_text_files(korean_txt_files)

print(f"\n로드된 전체 문서 수: {len(korean_data)}")
if korean_data:
    print(f"첫 번째 문서 내용 일부: {korean_data[0].page_content[:100]}...")


로딩 중: data\Rivian_KR.txt
로딩 중: data\Tesla_KR.txt

로드된 전체 문서 수: 2
첫 번째 문서 내용 일부: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당...


In [5]:
from transformers import AutoTokenizer
from langchain_text_splitters import CharacterTextSplitter

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장 구분자(정규식)를 사용하여 텍스트 분할기 생성
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,          # 토큰 수 계산 기준
    separator=r"[.!?]\s+",     # 문장 구분자: 마침표, 느낌표, 물음표 뒤 공백
    chunk_size=100,             # 청크 최대 토큰 수 (토크나이저 기준)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,    # separator가 정규식임을 명시
    keep_separator=True,        # 구분자 유지 여부
)

In [6]:
korean_docs = []
if korean_data: # 로드된 데이터가 있을 경우에만 분할 수행
    korean_docs = text_splitter.split_documents(korean_data)

print(f"\n분할된 한국어 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"첫 번째 분할 문서 내용: {korean_docs[0].page_content}")
    print(f"첫 번째 분할 문서 메타데이터: {korean_docs[0].metadata}")


분할된 한국어 문서 수: 8
첫 번째 분할 문서 내용: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다
첫 번째 분할 문서 메타데이터: {'source': 'data\\Rivian_KR.txt'}


In [7]:
import json 
final_docs_for_eval = []
if korean_docs: 
    for i, doc in enumerate(korean_docs):
        new_doc = doc.copy() 
        new_doc.metadata['doc_id'] = f'{i}'
        source_filename = os.path.basename(new_doc.metadata.get('source', ''))
        corp_name = source_filename.split('_')[0] if '_' in source_filename else source_filename.replace('.txt', '')
        new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 '{corp_name}'에 대한 정보를 담고 있습니다.)"
        final_docs_for_eval.append(new_doc)

    print(f"평가용으로 정제된 문서 수: {len(final_docs_for_eval)}")
    if final_docs_for_eval:
        print("\n첫 번째 정제 문서 예시:")
        print(final_docs_for_eval[0].page_content)
        print("-" * 30)
        print(final_docs_for_eval[0].metadata)
        print("=" * 30)

    with open('./data/final_docs_for_eval.jsonl', 'w', encoding='utf-8') as f:
        for doc_item in final_docs_for_eval:
            f.write(json.dumps({'page_content': doc_item.page_content, 'metadata': doc_item.metadata}) + '\n')
    print("\n평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.")

else:
    print("korean_docs가 없어 평가용 문서를 정제할 수 없습니다.")

평가용으로 정제된 문서 수: 8

첫 번째 정제 문서 예시:
2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다

(참고: 이 문서는 'Rivian'에 대한 정보를 담고 있습니다.)
------------------------------
{'source': 'data\\Rivian_KR.txt', 'doc_id': '0'}

평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.


C:\Users\USER\AppData\Local\Temp\ipykernel_29428\3445766471.py:5: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_doc = doc.copy()


`(2) Question-Answer (QA) 합성`

- 정제된 문서(`final_docs_for_eval`)의 각 청크(문서)를 컨텍스트로 하여, LLM(여기서는 OpenAI GPT 모델)을 사용해 질문과 답변 쌍을 생성함.
- `PydanticOutputParser`를 사용하여 LLM의 출력을 구조화된 객체(QAPair, QASet)로 파싱함.
- 프롬프트 엔지니어링을 통해 사실 기반의, 검색 엔진 사용자 스타일 질문을 생성하도록 유도함.
- **장점:** 대량의 QA 평가 데이터셋을 자동으로 생성 가능.
- **단점:**
  - LLM API 사용 비용 발생 (OpenAI API 키 필요).
  - 생성된 QA의 품질이 완벽하지 않을 수 있어 검토 및 수정 과정이 필요함.
  - 프롬프트에 민감하게 반응하므로, 원하는 품질의 QA를 얻기 위해 프롬프트 튜닝이 필요할 수 있음.

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import pandas as pd

# QA 쌍을 위한 Pydantic 모델 정의
class QAPair(BaseModel):
    question: str = Field(description="생성된 질문 (한국어로 작성)")
    answer: str = Field(description="질문에 대한 답변 (한국어로 작성, 질문의 핵심 내용을 반영)")

class QASet(BaseModel):
    qa_pairs: List[QAPair] = Field(description="질문-답변 쌍(QAPair)의 리스트")

# QA 생성 프롬프트 템플릿
QA_GENERATION_TEMPLATE_KOREAN = """
당신은 주어진 컨텍스트를 기반으로 사실에 입각한 질문-답변 쌍을 생성하는 AI입니다.
컨텍스트에서 특정하고 간결한 사실 정보로 답변할 수 있는 질문 {num_questions_per_chunk}개를 만드세요.
질문은 사용자가 검색 엔진에 질문할 법한 스타일로 작성해주세요.
질문에 "제시된 구절에 따르면" 또는 "컨텍스트에 따르면" 같은 문구는 포함하지 마세요.
답변에는 명확하고 완전한 정보를 제공하기 위해 질문의 핵심 내용이 포함되도록 하세요.

---------------------------------------------------------
출력은 다음 형식으로 제공해주세요:
{format_instructions}
---------------------------------------------------------
이제 컨텍스트를 제공합니다:
컨텍스트: {context}
"""

qa_llm = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=1000,
    temperature=0.2,
)

pydantic_parser_qa = PydanticOutputParser(pydantic_object=QASet)

qa_generation_prompt = ChatPromptTemplate.from_template(
    template=QA_GENERATION_TEMPLATE_KOREAN,
    partial_variables={"format_instructions": pydantic_parser_qa.get_format_instructions()}
)

qa_generation_chain = qa_generation_prompt | qa_llm | pydantic_parser_qa

# QA 생성 테스트 (첫 번째 정제 문서 사용)
if final_docs_for_eval:
    test_context_for_qa = final_docs_for_eval[0].page_content
    print(f"QA 생성 테스트용 컨텍스트:\n{test_context_for_qa}\n")
    
    qa_set_example = qa_generation_chain.invoke({
        "context": test_context_for_qa,
        "num_questions_per_chunk": 1 # 테스트로 1개만 생성
    })
    print("생성된 QA 쌍 예시:")
    for qa_pair in qa_set_example.qa_pairs:
        print(f"  질문: {qa_pair.question}")
        print(f"  답변: {qa_pair.answer}")
    print("실제 QA 생성은 주석 처리됨 (API 비용 발생 방지). 필요시 주석 해제 후 실행.")
else:
    print("정제된 평가용 문서가 없어 QA 생성 테스트를 스킵합니다.")

QA 생성 테스트용 컨텍스트:
2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다

(참고: 이 문서는 'Rivian'에 대한 정보를 담고 있습니다.)

생성된 QA 쌍 예시:
  질문: 리비안의 설립자는 누구인가요?
  답변: 리비안은 2009년 MIT 박사 과정생 RJ 스캐린지에 의해 설립되었습니다.
실제 QA 생성은 주석 처리됨 (API 비용 발생 방지). 필요시 주석 해제 후 실행.


In [9]:
# 전체 평가용 문서에 대해 QA 생성
NUM_QUESTIONS_PER_CHUNK = 2 # 각 문서 청크당 생성할 QA 개수
generated_qa_outputs = []

if final_docs_for_eval :
    print(f"{len(final_docs_for_eval)}개의 문서에 대해 QA 생성을 시작합니다...")
    for i, doc_item in enumerate(final_docs_for_eval):
        print(f"{i+1}/{len(final_docs_for_eval)}번째 문서 처리 중...", end=' ')
        try:
            qa_set_generated = qa_generation_chain.invoke({
                "context": doc_item.page_content,
                "num_questions_per_chunk": NUM_QUESTIONS_PER_CHUNK
            })
            for qa_pair in qa_set_generated.qa_pairs:
                generated_qa_outputs.append({
                    'context': [doc_item.page_content], # 정답 컨텍스트 (리스트 형태)
                    'source': [doc_item.metadata.get('source', '')], # 출처 (리스트 형태)
                    'doc_id': [doc_item.metadata.get('doc_id', '')], # 문서 ID (리스트 형태)
                    'question': qa_pair.question,
                    'answer': qa_pair.answer
                })
            print("성공")
        except Exception as e:
            print(f"실패: {e}")
            continue # 오류 발생 시 다음 문서로
    
    df_generated_qa_test = pd.DataFrame(generated_qa_outputs)
    print(f"\n총 {df_generated_qa_test.shape[0]}개의 QA 쌍이 생성되었습니다.")
    df_generated_qa_test.to_excel("./data/generated_qa_test.xlsx", index=False)
else:
    print("QA 합성은 실제 실행되지 않았습니다. (final_docs_for_eval이 없거나, 실행 플래그가 False임)")

8개의 문서에 대해 QA 생성을 시작합니다...
1/8번째 문서 처리 중... 성공
2/8번째 문서 처리 중... 성공
3/8번째 문서 처리 중... 성공
4/8번째 문서 처리 중... 성공
5/8번째 문서 처리 중... 성공
6/8번째 문서 처리 중... 성공
7/8번째 문서 처리 중... 성공
8/8번째 문서 처리 중... 성공

총 16개의 QA 쌍이 생성되었습니다.


In [10]:
df_generated_qa_test.head()

,context,source,doc_id,question,answer
0,[2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신...,[data\Rivian_KR.txt],[0],리비안은 언제 설립되었나요?,리비안은 2009년에 설립되었습니다.
1,[2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신...,[data\Rivian_KR.txt],[0],리비안의 본사는 어디에 위치하고 있나요?,리비안의 본사는 미시간주 리보니아에 위치하고 있습니다.
2,[.\n\n리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미...,[data\Rivian_KR.txt],[1],리비안의 초기 프로젝트는 어떤 차량이었나요?,리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미드십 엔진 ...
3,[.\n\n리비안의 초기 프로젝트는 피터 스티븐스가 디자인한 2+2 시트 배열의 미...,[data\Rivian_KR.txt],[1],R1 차량의 특징은 무엇인가요?,R1 차량은 모듈식 캡슐 구조와 쉽게 교체 가능한 차체 패널을 특징으로 합니다.
4,"[. 리비안은 디젤 하이브리드, 브라질 원메이크 시리즈를 위한 R1 GT라는 이름의...",[data\Rivian_KR.txt],[2],리비안의 R1 GT는 어떤 종류의 차량인가요?,"R1 GT는 리비안의 레이싱 버전으로, 디젤 하이브리드 차량입니다."


`(3) 테스트 데이터 검토 및 수정`

- LLM이 생성한 QA 데이터는 완벽하지 않을 수 있으므로, 사람이 직접 검토하고 수정하는 과정이 중요함.
- **장점:** 평가 데이터의 품질을 크게 향상시켜 평가 결과의 신뢰도를 높임.
- **단점:** 시간과 노력이 많이 소요되는 수동 작업임.

### 1.2 Information Retrieval (IR) 평가지표

- 검색 시스템이 얼마나 관련 있는 문서를 잘 찾아내는지, 그리고 그 순서는 적절한지를 평가하는 지표
- 주요 지표: Hit Rate, MRR, mAP@k, NDCG@k 등.

In [11]:
from langchain_core.documents import Document

# 각 쿼리에 대한 정답 문서 (실제 관련 문서)
actual_docs_sample = [
    # Query 1에 대한 정답
    [Document(metadata={'id': 'doc1'}, page_content='내용1')],
    # Query 2에 대한 정답
    [Document(metadata={'id': 'doc2'}, page_content='내용2'), Document(metadata={'id': 'doc5'}, page_content='내용5')]
]

# 각 쿼리에 대한 검색기가 예측한(찾아온) 문서
predicted_docs_sample = [
    # Query 1에 대한 예측
    [Document(metadata={'id': 'doc1'}, page_content='내용1'), Document(metadata={'id': 'doc3'}, page_content='내용3')],
    # Query 2에 대한 예측
    [Document(metadata={'id': 'doc4'}, page_content='내용4'), Document(metadata={'id': 'doc1'}, page_content='내용1'), 
     Document(metadata={'id': 'doc5'}, page_content='내용5'), Document(metadata={'id': 'doc2'}, page_content='내용2')]
]

print("샘플 정답 문서 (actual_docs_sample) 및 예측 문서 (predicted_docs_sample) 준비 완료.")

샘플 정답 문서 (actual_docs_sample) 및 예측 문서 (predicted_docs_sample) 준비 완료.


**평가 지표 설명 (개념)**

`(1) Hit Rate@k`
- 정의: 각 쿼리에 대해 검색된 상위 `k`개의 문서 중에 실제 정답 문서가 **하나라도 포함**되어 있으면 1 (Hit), 아니면 0으로 계산. 전체 쿼리에 대한 평균을 냄.
- 특징: 간단하고 직관적이지만, 정답 문서의 순위나 개수는 고려하지 않음.
- 예시 (k=2, Query 1): 예측 [`doc1`, `doc3`], 정답 [`doc1`]. `doc1`이 포함되어 Hit (1).
- 예시 (k=2, Query 2): 예측 [`doc4`, `doc1`], 정답 [`doc2`, `doc5`]. `doc2` 또는 `doc5` 미포함 (정답 문서 ID 기준). Not Hit (0). (샘플 데이터에서는 `doc5`가 3번째, `doc2`가 4번째에 있음)
- Hit Rate@2 = (1 + 0) / 2 = 0.5 (만약 Query 2 예측에 `doc5`나 `doc2`가 상위 2개 안에 있었다면 1)

`(2) MRR@k (Mean Reciprocal Rank)`
- 정의: 각 쿼리에 대해 **첫 번째 정답 문서**가 검색된 순위(rank)의 역수(1/rank)를 계산. 이 값들의 평균.
- 특징: 사용자가 원하는 결과를 얼마나 빨리 찾을 수 있는지(첫 정답의 순위)를 평가. 상위 `k`개 내에서만 고려.
- 예시 (k=4, Query 1): 예측 [`doc1`, `doc3`], 정답 [`doc1`]. `doc1`이 1등. RR = 1/1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`, `doc1`, `doc5`, `doc2`], 정답 [`doc2`, `doc5`]. 첫 정답 `doc5`가 3등. RR = 1/3.
- MRR@4 = (1 + 1/3) / 2 ≈ 0.667.

`(3) mAP@k (Mean Average Precision at k)`
- 정의: 각 쿼리에 대한 AP@k(Average Precision at k)를 구하고, 이들의 평균을 냄. AP@k는 상위 `k`개 결과 내에서 각 정답 문서가 나올 때마다의 Precision(정밀도) 값들의 평균.
- 특징: 검색 결과의 순서와 정확도를 모두 고려. 여러 정답이 있는 경우 유용.
- Precision@i: 상위 i개 결과 중 정답 문서의 비율.
- AP@k 계산: 각 정답 문서 위치 `j` (j <= k)에서의 Precision@j 값을 모두 더한 후, 총 정답 문서 수 (또는 k개 내에 있는 정답 문서 수)로 나눔.
- 예시 (k=4, Query 1): 예측 [`doc1`(정답), `doc3`], 정답 [`doc1`]. 
  - P@1 (`doc1`): 1/1 = 1. AP@4_Q1 = (1) / 1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`, `doc1`, `doc5`(정답), `doc2`(정답)], 정답 [`doc2`, `doc5`].
  - P@1 (`doc4`): 0/1=0.
  - P@2 (`doc1`): 0/2=0.
  - P@3 (`doc5`): 1/3. (첫 정답)
  - P@4 (`doc2`): 2/4. (두 번째 정답)
  - AP@4_Q2 = (1/3 + 2/4) / 2 = (0.333 + 0.5) / 2 = 0.4165.
- mAP@4 = (1 + 0.4165) / 2 ≈ 0.708.

`(4) NDCG@k (Normalized Discounted Cumulative Gain at k)`
- 정의: 검색 결과의 순서와 문서의 관련성 등급을 모두 고려하여 평가. 이상적인 검색 결과(IDCG) 대비 현재 검색 결과(DCG)의 비율.
- 특징: 관련성이 높은 문서가 상위에 있을수록 높은 점수. 가장 정교한 지표 중 하나.
- DCG@k = Σ ( (2^relevance_i - 1) / log2(i+1) )  (i는 순위 1부터 k까지, relevance_i는 i번째 문서의 관련도 점수)
- IDCG@k: 이상적인 순서일 때의 DCG@k (정답 문서들을 관련도 순으로 정렬했을 때).
- NDCG@k = DCG@k / IDCG@k.
- (이진 관련성 가정: 정답이면 1, 아니면 0)
- 예시 (k=4, Query 1): 예측 [`doc1`(rel=1), `doc3`(rel=0)], 정답 [`doc1`].
  - DCG@4_Q1 = (2^1-1)/log2(1+1) + (2^0-1)/log2(2+1) = 1/1 + 0 = 1.
  - IDCG@4_Q1 = (2^1-1)/log2(1+1) = 1.
  - NDCG@4_Q1 = 1/1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`(0), `doc1`(0), `doc5`(1), `doc2`(1)], 정답 [`doc2`, `doc5`]. (이상적 순서는 `doc2`(1), `doc5`(1) 또는 그 반대)
  - DCG@4_Q2 = 0/log2(2) + 0/log2(3) + 1/log2(4) + 1/log2(5) = 0 + 0 + 1/2 + 1/2.32 = 0.5 + 0.43 = 0.93.
  - IDCG@4_Q2 = 1/log2(2) + 1/log2(3) = 1 + 1/1.58 = 1 + 0.63 = 1.63.
  - NDCG@4_Q2 = 0.93 / 1.63 ≈ 0.57.
- 평균 NDCG@4 = (1 + 0.57) / 2 = 0.785.